In [36]:
# start docker deamon
# sudo docker login

In [76]:
!nextflow info

  Version: 23.04.4 build 5881
  Created: 25-09-2023 15:34 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.16 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [77]:
!gcloud config list 

[compute]
region = us-central1
zone = us-central1-a
[core]
account = eila@stanford.edu
disable_usage_reporting = True
project = hai-gcp-genomic

Your active configuration is: [default]


In [ ]:
# I will need to login to a different project for the Anvil

In [79]:
# for the service account:
# https://www.nextflow.io/docs/latest/google.html
# cloud auth application-default login
# the credentials will be saved to [/home/eila/.config/gcloud/application_default_credentials.json] for 
# the use of the application

# https://github.com/nextflow-io/nextflow/issues/1709
!export NXF_VER="23.04.4"
!export NXF_MODE=google
!export NXF_DEBUG=3
# TODO: change for the Anvil
!export PROJECT="hai-gcp-genomic"
!export GOOGLE_APPLICATION_CREDENTIALS=/home/eila/.config/gcloud/application_default_credentials.json

In [80]:
!cat /home/eila/.config/gcloud/application_default_credentials.json

{
  "client_id": "764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com",
  "client_secret": "d-FL95Q19q7MQmFpd7hHD0Ty",
  "quota_project_id": "hai-gcp-genomic",
  "refresh_token": "1//04-ZrKvCfXPa5CgYIARAAGAQSNwF-L9IrkFtcwyfUDAhTeAquw0t0utu5nMRfG-U9l5uflp_XJI0EMWlO9aPKmi0CxdbKiKRekQc",
  "type": "authorized_user",
  "project_id": "hai-gcp-genomic"
}


In [81]:
!nextflow clean -f nf_wf_fg_singlecell_atac.nf

In [82]:
import os
os.getcwd()

'/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac'

In [107]:
%%writefile nextflow.config

includeConfig 'conf/conda.config'


manifest {
  description = 'A nexflow pipeline for fg single cell atac' 
  mainScript = 'nf_wf_fg_singlecell_atac.nf'
}

profiles {
  conda_profile {
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
    }

  conda_docker_profile {
    includeConfig 'conf/conda_docker.config'
    executor = 'local'
    }

  docker_profile {
    includeConfig 'conf/docker.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'local'
  }
  google_batch_profile {
    workDir = 'gs://eila-nextflow-bucket/work'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'google-batch'
    google.project = 'hai-gcp-genomic'
    location = 'us-central1'
    batch.spot = false
  }
    
}

params.FASTQS_SPEC_CH = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_local.csv'
params.CHROMAP_IDX='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/Homo_sapiens.GRCh38.dna.primary_assembly.chromap.ref.index'

params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'
params.OUTDIR='./igvf-out'



Overwriting nextflow.config


In [108]:
!pwd

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac


In [109]:
!rm -r /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/work

rm: cannot remove '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/work': No such file or directory


In [110]:
!rm .nextflow*

rm: cannot remove '.nextflow': Is a directory


In [111]:
!ls nextflow.config

nextflow.config


In [112]:

# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow.config -profile docker_profile
!nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow.config -profile google_batch_profile

N E X T F L O W  ~  version 23.04.4
ERROR ~ Unknown method invocation `echo` on _parse_closure5 type

 -- Check '.nextflow.log' file for details


In [105]:
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow.config -profile google_batch_profile -resume

In [57]:
# !ls /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/seqspec/specs/dogmaseq-dig/fastqs

In [ ]:
# !nextflow run /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_processes/nf_prcs_print.nf

In [ ]:
# !ls /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_processes/nf_prcs_print.nf

In [ ]:
# !cat .command.out

In [ ]:
# !nextflow run nf_wf_fg_singlecell.nf -c nextflow.config -with-docker eilalan/env_synapse:latest